### TSP formulation: 

N = # number of cities to visit ( 0 represents depot )

I = set of cities = {0,...,N}

K = set of cities excluding depot = {1,...,N}

$v_{i} = $ each city i visited in order excluding depot.

$d_{ij} = $ distance between city i to city j.

$X_{ij} $ $ = 1 $ if city j is visited from city i.

Model

MIN $ Z = \sum_{i=0}^n \sum_{j=0}^n {d_{ij} * X_{ij}}$

subject to:
    
1)  reach every city from exactly one predecessor

    $\sum_{i=0}^n {X_{ij}} = 1 $ $\forall j$ $\in$ I

2)  leave every city to exactly one succesor

    $\sum_{j=0}^n {X_{ij}} = 1 $ $\forall i$ $\in$ I

3) subtour elimination

    $ (N - 1)*(1 - X_{ij}) $ $ \geq $ $ v_{i}  - v_{j} + 1  $   $\forall i, j$ $\in$ K

4) variable types: x as binary, v as integer

    $ X_{ij} $ $\in$ {0,1} ,  $ v_{i} $ $\in$ { 0, ... , N - 1 }

    

In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp
import time
import numpy as np
import copy

#distance matrix of cities, row are from, cols are to, value of row and col is distance from row to col city
distance_matrix = pd.DataFrame([
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
])


def tsp_math_model(d_ij:pd.DataFrame):
    """
    Returns route with min total distance considering round trip. (TSP math model)
    """
    # method run time
    start_time = time.time()

    # number of cities
    N = len(d_ij)

    # set of cities
    I = [*range(0,N)]

    # set of cities excluding depot
    K = [*range(1,N)]

    # Solver
    solver = pywraplp.Solver("TSP",pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    # Variable: t[i] indicates the sequential order of visit and takes values from 1 to N - 1
    v = {}
    for i in K:
        v[i] = solver.IntVar(1, N-1, 't[%i]'%(i))

    # Variable: x[i, j] as binary 0 or 1
    x = {}
    for i in I:
        for j in I:
            x[i, j] = solver.BoolVar('x[%i,%i]'%(i,j))

    # Objective: min total distance traveled
    solver.Minimize(solver.Sum([d_ij.iloc[i,j]*x[i, j] for i in I for j in I]))

    # Constraint 1: exactly one predecessor
    for j in I:
        solver.Add(solver.Sum([x[i, j] for i in I]) == 1)


    # Constraint 2: exactly one successor
    for i in I:
        solver.Add(solver.Sum([x[i, j] for j in I]) == 1)

    # Constraint 3 : Subtour elimination
    for i in K:
        for j in K:
            solver.Add( v[i] - v[j] + 1  <= (N - 1)*(1-x[i,j]))


    # Print solution
    sol = solver.Solve()
    if sol == pywraplp.Solver.OPTIMAL:
        print('OF: Z = ', solver.Objective().Value())
        for i in range(N):
            for j in range(N):
                if x[i,j].solution_value() > 0:
                    print('x(%d,%d) = %.2f' %(i,j,x[i,j].solution_value()))

    # print run time
    print("--- %s PARS model run time in seconds ---" % (time.time() - start_time))

# Driver code
if __name__ == '__main__':
    tsp_math_model(d_ij = distance_matrix)

/Users/adames/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


OF: Z =  7293.0
x(0,9) = 1.00
x(1,8) = 1.00
x(2,7) = 1.00
x(3,2) = 1.00
x(4,3) = 1.00
x(5,10) = 1.00
x(6,12) = 1.00
x(7,0) = 1.00
x(8,6) = 1.00
x(9,5) = 1.00
x(10,11) = 1.00
x(11,1) = 1.00
x(12,4) = 1.00
--- 63.00591683387756 PARS model run time in seconds ---


### RL for TSP

In [3]:
def update_q(q, distance_matrix, state, action, alpha, gamma):
    ''' Updates Q table for reinforcement learning'''

    # reward for current stage and action
    reward =  1/distance_matrix.loc[state,action]

    # max reward for next stage and activity value
    delayed_reward = q[action,:].max()

    # update q value
    q[state,action] = q[state,action] + alpha * (reward + gamma * delayed_reward -  q[state,action])

    return q
    
def encoding(string:str, initial_city:int, distance_matrix:pd.DataFrame):
    ''' Return objective function for vehicle routing problem given a string of sequences where the delimeter | indicates a new vehicle'''

    # get cities in sequence
    l = [ int(x) for x in string.split('-') ] 
    l.insert(0,initial_city)
    l.insert(len(l),initial_city)

    # get distance traveled
    total_distance = []
    for i in range(len(l)):
        if i < len(l)-1:
            total_distance.append(distance_matrix.iloc[l[i], l[i+1]])

    # store total distances traveled by vehicle and total distance among all vehicles
    obj_val = sum(total_distance)

    return  obj_val

def reinforcement_learning(distance_matrix, exploration_proba, n_episodes, alpha, gamma):
    '''Main call for RL appied to TSP'''
    # method run time
    start_time = time.time()
    
    # initializing cities and q matrix
    cities = len(distance_matrix)
    q = np.zeros([cities,cities])

    # store values
    seqs, Z = {} , {}

    # exploration thresholds
    exploration_decreasing_decay , min_exploration_proba = 0.001 , 0.01

    # train model to learn from experience
    for e in range(n_episodes):
        # initial city as initial stat
        visited, state = [0] , 0

        # posible actions at initial state
        possible_actions = [ next_city for next_city in range(cities) if next_city not in visited]

        # iterate until all destinations are visited
        while possible_actions:
            if np.random.uniform(0,1) < exploration_proba:
                action = np.random.choice(possible_actions)
            else:       
                best_action_index = q[state, possible_actions].argmax()
                action = possible_actions[best_action_index]

            # update q table
            q = update_q(q, distance_matrix, state, action,alpha=alpha, gamma=gamma)
            visited.append(action)
            state = visited[-1]
            possible_actions = [ next_city for next_city in range(cities) if next_city not in visited]

        #We update the exploration proba using exponential decay formula 
        exploration_proba = max(min_exploration_proba, np.exp(-exploration_decreasing_decay*e)) 
        seqs[e] = '-'.join([str(y) for y in copy.copy(visited) if y!=0 ] ) #visited
        Z[e] = encoding(string = '-'.join([str(y) for y in copy.copy(visited) if y!=0 ] ), initial_city = 0,  distance_matrix = distance_matrix)
    
    # best sequence
    z,s = Z[min(Z, key=Z.get)], seqs[min(Z, key=Z.get)]
    
    #Print Results:
    print('Best trajectory found:')
    print(' -> '.join([ str(x) for x in (str(0)+'-'+s+'-'+str(0)).split('-') ] ))

    print('OF: ', z)

    # print run time
    print("--- %s PARS model run time in seconds ---" % (time.time() - start_time))
    return z, s, Z, seqs


# RL call
alpha_grid = [0.1, 0.5, 0.99]
gamma_grid = [0.1, 0.5, 0.99]

for a in alpha_grid:
    for y in gamma_grid:
        reinforcement_learning(distance_matrix = distance_matrix, exploration_proba = 1, n_episodes = 50000, alpha=a, gamma=y)
        
#z, s, Z, seqs = reinforcement_learning(distance_matrix = distance_matrix, exploration_proba = 1, n_episodes = 50000, alpha=0.99, gamma=0.5)

Best trajectory found:
0 -> 7 -> 2 -> 9 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 5 -> 10 -> 0
OF:  7534
--- 37.593788862228394 PARS model run time in seconds ---
Best trajectory found:
0 -> 7 -> 3 -> 2 -> 9 -> 5 -> 10 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 0
OF:  8259
--- 44.055431842803955 PARS model run time in seconds ---
Best trajectory found:
0 -> 2 -> 9 -> 10 -> 5 -> 11 -> 8 -> 1 -> 6 -> 12 -> 4 -> 3 -> 7 -> 0
OF:  7889
--- 50.65460205078125 PARS model run time in seconds ---
Best trajectory found:
0 -> 7 -> 2 -> 9 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 5 -> 10 -> 0
OF:  7534
--- 42.0099778175354 PARS model run time in seconds ---
Best trajectory found:
0 -> 7 -> 5 -> 10 -> 4 -> 12 -> 11 -> 1 -> 8 -> 6 -> 3 -> 2 -> 9 -> 0
OF:  8068
--- 41.00327491760254 PARS model run time in seconds ---
Best trajectory found:
0 -> 2 -> 9 -> 10 -> 5 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 3 -> 7 -> 0
OF:  7668
--- 40.522210121154785 PARS model run time in seconds ---
Best trajectory found:
0 -> 7 -> 2 -> 